# Reinforcement Learning Solution to the Towers of Hanoi Puzzle

In [486]:
import neuralnetworks as nnQ
import random
import numpy as np

In [7]:
nnetQ = nnQ.NeuralNetwork(2, 3, 1)

For this assignment, you will use reinforcement learning to solve the [Towers of Hanoi](https://en.wikipedia.org/wiki/Tower_of_Hanoi) puzzle.  

To accomplish this, you must modify the code discussed in lecture for learning to play Tic-Tac-Toe.  Modify the code  so that it learns to solve the three-disk, three-peg
Towers of Hanoi Puzzle.  In some ways, this will be simpler than the
Tic-Tac-Toe code.  

Steps required to do this include the following:

  - Represent the state, and use it as a tuple as a key to the Q dictionary.
  - Make sure only valid moves are tried from each state.
  - Assign reinforcement of $1$ to each move, even for the move that results in the goal state.

Make a plot of the number of steps required to reach the goal for each
trial.  Each trial starts from the same initial state.  Decay epsilon
as in the Tic-Tac-Toe code.

## Requirements

First, how should we represent the state of this puzzle?  We need to keep track of which disks are on which pegs. Name the disks 1, 2, and 3, with 1 being the smallest disk and 3 being the largest. The set of disks on a peg can be represented as a list of integers.  Then the state can be a list of three lists.

For example, the starting state with all disks being on the left peg would be `[[1, 2, 3], [], []]`.  After moving disk 1 to peg 2, we have `[[2, 3], [1], []]`.

To represent that move we just made, we can use a list of two peg numbers, like `[1, 2]`, representing a move of the top disk on peg 1 to peg 2.

Now on to some functions. Define at least the following functions. Examples showing required output appear below.

   - `printState(state)`: prints the state in the form shown below
   - `validMoves(state)`: returns list of moves that are valid from `state`
   - `makeMove(state, move)`: returns new (copy of) state after move has been applied.
   - `trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF)`: train the Q function for number of repetitions, decaying epsilon at start of each repetition. Returns Q and list or array of number of steps to reach goal for each repetition.
   - `testQ(Q, maxSteps, validMovesF, makeMoveF)`: without updating Q, use Q to find greedy action each step until goal is found. Return path of states.

A function that you might choose to implement is

   - `stateMoveTuple(state, move)`: returns tuple of state and move.  
    
This is useful for converting state and move to a key to be used for the Q dictionary.

Show the code and results for testing each function.  Then experiment with various values of `nRepetitions`, `learningRate`, and `epsilonDecayFactor` to find values that work reasonably well, meaning that eventually the minimum solution path of seven steps is found consistently.

Make a plot of the number of steps in the solution path versus number of repetitions. The plot should clearly show the number of steps in the solution path eventually reaching the minimum of seven steps, though the decrease will not be monotonic.  Also plot a horizontal, dashed line at 7 to show the optimal path length.

Add markdown cells in which you describe the Q learning algorithm and your implementation of Q learning as applied to the Towers of Hanoi problem.  Use at least 15 sentences, in one or more markdown cells.

In [462]:
def stateMoveTuple(state, move):
    tempTupState = []
    for x in state:
        tempTupState.append(tuple(x))
        
    return (tuple(tempTupState),tuple(move))

In [557]:
def trainQ(nRepetitions, learningRate, epsilonDecayFactor, validMovesF, makeMoveF):
    Q = {}
    state = [[1,2,3],[],[]]
    #initialize Q to empty dictionary
    #set state = start state
    epsilon = 1
    n = 0
    for x in range(nRepetitions):
        epsilon = epsilonDecayFactor * epsilon
        step_count = []
        step = 0

        goal_state = [[],[],[1,2,3]]
        done = False
        
        while not done and (step <nRepetitions):
            next_move = epsilonFindMoves(Q, state, epsilonDecayFactor, validMovesF)
            next_state = makeMove(state,next_move) 
            #to make the next move
            #print(next_state,'=',goal_state)
            if next_state == goal_state:
                #print('here')
                Q[stateMoveTuple(next_state,next_move)] = 1
                #tempQ = {stateMoveTuple(state,next_move),1}
                #Q.update(tempQ)
                
                done = True
                  #Update Q at (state,move) to equal 1
            else:
                  #Initialize Q at (state, move) to equal 0
                Q[stateMoveTuple(next_state,next_move)] = 0
                Q[stateMoveTuple(next_state,next_move)] = Q.get(stateMoveTuple(next_state,next_move),0) + learningRate *(
                                                        1+Q.get(stateMoveTuple(next_state,next_move),0)-
                                                        Q.get(stateMoveTuple(next_state,next_move),0))
                #Q[state, move] = Q[state, move] + learningRate * (1+Q[next_state, next_move] - Q[state, move])
                #increment step count
                step = step+1
            step_count.insert(0, step)
    return Q ,step_count

In [558]:
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)

In [559]:
stepsToGoal

[8, 8, 7, 6, 5, 4, 3, 2, 1]

In [531]:
def printState(state):

    temp = 0
    col1Len = 0
    col2Len = 0
    col3Len = 0
    big = len(max(state))
    while temp != len(max(state)):
        holder1,holder2,holder3 = " "," "," "
        if col1Len< len(state[0]) and col1Len !=big and col1Len == temp:
            holder1 = state[0][temp]
            col1Len = col1Len + 1
        if col2Len < len(state[1]) and col2Len == temp:
            holder2 = state[1][temp]
            col2Len = col2Len + 1
        if col3Len < len(state[2])and col3Len == temp:
            holder3 = state[2][temp]
            col3Len = col3Len + 1
        print(holder1,holder2,holder3)
                
        temp = temp+1
    print("------")


# Examples

In [319]:
state = [[1], [], [2,3]]
printState(state)

1   2
    3
------


In [368]:
def validMoves(state):
    validMoves = []
    for x in range(1,4):
        for y in range(1,4):
            if len(state[x-1]) > 0:
                topx = state[x-1]
                if len(state[y-1]) !=0:
                    topy = state[y-1]
                if len(state[y-1]) == 0:
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])
                elif((topx != topy and topy > topx)):
                    validMoves.append([x,y])

    return validMoves                

In [456]:
def makeMove(state, move):
    temp = state[move[0]-1][0]
    state[move[0]-1].pop(0)
    state[move[1]-1].insert(0,temp)
    state2 = state
    return state2

In [475]:
def epsilonGreedy(nnetQ, state, validActions, epsilon):
    if np.random.uniform() < epsilon:
        # Random Move
        action = np.random.choice(validActions)
    else:
        # Greedy Move
        actionsRandomlyOrdered = random.sample(validActions, len(validActions))
        Qs = [nnetQ.use(np.array([[state,a]])) for a in actionsRandomlyOrdered]
        ai = np.argmax(Qs)
        action = actionsRandomlyOrdered[ai]
    Q = nnetQ.use(np.array([[state,action]]))
    return action, Q

def epsilonFindMoves(Q, state, epsilonRate, validMoveF):
    validMoveList = validMoveF(state)
    if np.random.uniform()<epsilonRate:
        return random.choice(validMoveList)
    else:
        Qs = np.array([Q.get(stateMoveTuple(state, m), 0) for m in validMoveList])
        return validMoveList[np.argmin(Qs)]

In [458]:
state = [[1,2,3],[],[]]
move = [1,2]
print(makeMove([[2], [3], [1]], [1, 2]))

2
[[], [2, 3], [1]]
[[], [2, 3], [1]]


In [346]:
state = [[1,2,3],[],[]]
print(validMoves(state))

[[1, 2], [1, 3]]


In [329]:
state = [[1, 2, 3], [], []]
printState(state)

1    
2    
3    
------


In [3]:
move =[1, 2]

stateMoveTuple(state, move)

(((1, 2, 3), (), ()), (1, 2))

In [4]:
newstate = makeMove(state, move)
newstate

[[2, 3], [1], []]

In [5]:
printState(newstate)

2     
3 1   
------



In [496]:
Q, stepsToGoal = trainQ(50, 0.5, 0.7, validMoves, makeMove)

KeyboardInterrupt: 

In [7]:
stepsToGoal

array([ 178.,   23.,   77.,   60.,   14.,    7.,   36.,   28.,    9.,
          7.,   10.,   17.,   44.,    8.,   16.,    7.,   22.,    7.,
         11.,    7.,   10.,    7.,    7.,    7.,    7.,    7.,    7.,
          7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,
          7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,    7.,
          7.,    7.,    7.,    7.,    7.])

In [8]:
path = testQ(Q, 20, validMoves, makeMove)

In [9]:
path

[[[1, 2, 3], [], []],
 [[2, 3], [], [1]],
 [[3], [2], [1]],
 [[3], [1, 2], []],
 [[], [1, 2], [3]],
 [[1], [2], [3]],
 [[1], [], [2, 3]],
 [[], [], [1, 2, 3]]]

In [10]:
for s in path:
    printState(s)
    print()

1     
2     
3     
------


2     
3   1 
------


3 2 1 
------


  1   
3 2   
------


  1   
  2 3 
------


1 2 3 
------


    2 
1   3 
------


    1 
    2 
    3 
------




## Grading

Download and extract `A4grader.py` from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar).

In [560]:
%run -i A4grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Armijo-A4.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.
CRITICAL ERROR: Function named 'testQ' is not defined
  Check the spelling and capitalization of the function name.

Testing validMoves([[1], [2], [3]])

--- 10/10 points. Correctly returned [[1, 2], [1, 3], [2, 3]]

Testing validMoves([[], [], [1, 2, 3]])

--- 10/10 points. Correctly returned [[3, 1], [3, 2]]

Testing makeMove([[], [], [1, 2, 3]], [3, 2])

--- 10/10 points. Correctly returned [[], [1], [2, 3]]

Testing makeMove([[2], [3], [1]], [1, 2])

--- 10/10 points. Correctly returned [[], [2, 3], [1]]

Testing   Q, steps = trainQ(1000, 0.5, 0.7, validMoves, makeMove).

--- 10/10 points. Q dictionary has correct number of entries.

--- 10/10 points. The mean of the number of steps of 1.0 is correctly < 10.

Testing   path = testQ(Q, 20, validMov

## Extra Credit

Modify your code to solve the Towers of Hanoi puzzle with 4 disks instead of 3.  Name your functions

    - printState_4disk
    - validMoves_4disk
    - makeMove_4disk

Find values for number of repetitions, learning rate, and epsilon decay factor for which trainQ learns a Q function that testQ can use to find the shortest solution path.  Include the output from the successful calls to trainQ and testQ.